In [ ]:
import requests
from dotenv import load_dotenv
import os
import faiss
from sentence_transformers import SentenceTransformer

load_dotenv()

API_KEY = os.getenv("CLARIN_API_KEY")

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

def get_models():
    headers = {
        'Authorization': f'Bearer {API_KEY}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json()  

models = get_models()
print(models)


{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus'}, {'id': 'mixtral-8x22B', 'full_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'name': 'mistralai/Mixtral-8x22B-Instruct-v0.1'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210'}, {'id': 'pllum-shparag-llama3-8B', 'full_name': 'pllum/rag_70k_nemotron_finetun

In [ ]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

In [ ]:
def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"] 
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")

    index = faiss.IndexFlatL2(embeddings.shape[1])  
    index.add(embeddings)  
    return index, reviews


In [ ]:
def search_faiss(query, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_embedding, k)

    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews


In [ ]:
def clarin_chat_completion(model_id, prompt, max_tokens=200):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [23]:
def handle_query_with_rag(user_query, model_id="bielik", k=5):
    distances, result_reviews = search_faiss(user_query, k)

    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(result_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response


In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
index, reviews = build_faiss_index(dataset, model)

In [15]:
faiss.write_index(index, 'index/imdb_faiss.bin')

In [19]:
user_query = "What is the best movie to watch with friends?"
response = handle_query_with_rag(user_query)
print("Odpowiedź:", response)

Odpowiedź: Wybór najlepszego filmu do oglądania z przyjaciółmi zależy od kilku czynników, takich jak gust grupy, okazja oraz tematyka filmu. Ze swojej perspektywy mogę polecić kilka tytułów, które nadają się znakomicie na seans w towarzystwie znajomych.

Jeśli szukasz lekkiego i pełnego humoru filmu, świetną opcją będą komedie romantyczne, takie jak "Notting Hill" czy "Dziewczyna z sąsiedztwa". Oba te filmy nie tylko zapewnią sporo śmiechu, ale także stworzą miłą atmosferę do rozmowy po projekcji.

Dla fanów animacji polecam "Toy Story", serię filmów o Toy Story jest niezwykle uroczym i uniwersalnym wyborem, który spodoba się widzom w każdym wieku. Filmy te mają znakomitą fabułę oraz piękne przesłanie, a ich różnorodność sprawi, że nikt się nie znudzi.

Miłośnicy kina akcji mogą cieszyć się przygodami w cyklu "Mad Max" - te niezwykle dynamiczne i pełne spektakularnych efektów specjalnych produkcje zapewnią mocne wrażenia i dodadzą energii do spotkania ze znajomymi.

A jeśli chcesz wpro

In [24]:
handle_query_with_rag("What do people think about horror movies?")

'W oparciu o podane informacje, nie możemy bezpośrednio określić dokładnej opinii ludzi o horrorach, ponieważ zamiast treści recenzji mamy tylko numery referencyjne. Jednakże na ich podstawie możemy sformułować następujące domysły:\n\n1. Skoro są to recenzje filmów grozy, prawdopodobnie ludzie mają różnorodne opinie - od negatywnych po entuzjastyczne. \n\n2. Numery recenzji sugerują, że opinie byłyby rozłożone na kilka tysięcy widzów (np. dla recenzji 23708 - prawie 24 tysiące). W tak dużej grupie spodziewalibyśmy się również pogłębionej różnorodności preferencji i emocji wobec horrorów.\n\n3. Możliwe, że niektóre z tych recenzji dotyczą klasyków gatunku jak "Psychoza" czy "Piła", inne mogłyby omawiać najnowsze produkcje lub niszowe kino grozy.\n\nAby uzyskać szczegółową opinię publiczną na temat horrorów, potrzebne byłyby pełne treści tych recenzji oraz dodatkowe dane ze sprzedaży biletów, liczby wyświetleń czy reakcji w mediach społecznościowych. Jednocześnie pamiętajmy, że opinie os

______________

In [ ]:
import requests
from dotenv import load_dotenv
import os
import faiss
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline

# Wczytanie API Key dla CLARIN
load_dotenv()
API_KEY = os.getenv("CLARIN_API_KEY")

# Ustawienia dla CLARIN API
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"

# FAISS GPU Check
def initialize_faiss_on_gpu(index, quantizer=None):
    if torch.cuda.is_available():
        res = faiss.GpuResources()
        if quantizer:
            quantizer = faiss.index_cpu_to_gpu(res, 0, quantizer)
        index = faiss.index_cpu_to_gpu(res, 0, index)
    return index

# Funkcja do pobierania modeli z CLARIN API
def get_models():
    headers = {'Authorization': f'Bearer {API_KEY}', 'Content-Type': 'application/json'}
    response = requests.get(f"{CLARIN_BASE_URL}/models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch models: {response.status_code} - {response.text}")

# Pobranie IMDB Dataset
from datasets import load_dataset
dataset = load_dataset("stanfordnlp/imdb")

# FAISS z IndexIVFFlat
import faiss
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

# Funkcja do budowy indeksu FAISS z użyciem Product Quantization (IndexIVFPQ)
def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"]
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")
    
    d = embeddings.shape[1]  # Wymiar danych
    quantizer = faiss.IndexFlatL2(d)  # Klasteryzator

    # Tworzymy indeks IVFPQ z 100 klastrami, m=8 (liczba części kwantyzacji), nbits=8 (rozmiar każdej części)
    nlist = 100  # Liczba klastrów
    m = 8  # Liczba części kwantyzacji
    nbits = 8  # Liczba bitów przypisanych do każdej części
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)  # Tworzenie indeksu IVFPQ

    # Sprawdzenie, czy FAISS został przeniesiony na GPU
    index = initialize_faiss_on_gpu(index)  # Funkcja do przenoszenia na GPU (opcjonalna)

    index.train(embeddings)  # Trening FAISS na wektorach
    index.add(embeddings)  # Dodanie wektorów do indeksu

    # Zapisz indeks na dysk
    faiss.write_index(index, "index/imdb_faiss_ivfpq.bin")
    print("Indeks został zapisany na dysk.")

    return index, reviews


# Funkcja wyszukiwania FAISS
def search_faiss(index, query, model, reviews, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_embedding, k)
    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews

# Reranker z Cross-Encoder
def rerank_with_cross_encoder(query, contexts):
    reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    rerank_scores = reranker.predict([(query, context) for context in contexts])
    sorted_indices = sorted(range(len(rerank_scores)), key=lambda i: rerank_scores[i], reverse=True)
    return [contexts[i] for i in sorted_indices], rerank_scores

# Chat Completion z CLARIN
def clarin_chat_completion(model_id, prompt, max_tokens=200):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

# Funkcja obsługi zapytania z RAG
def handle_query_with_rag(user_query, model_id="bielik", k=5):
    distances, result_reviews = search_faiss(index, user_query, model, reviews, k)

    # Reranking
    reranked_reviews, rerank_scores = rerank_with_cross_encoder(user_query, result_reviews)

    # Przygotowanie kontekstu
    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(reranked_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response, context

# Ocena odpowiedzi modelem LLaMA
def evaluate_response_with_llama(question, context, answer):
    evaluator = pipeline("text-classification", model="meta-llama/LLaMA-2-7b-hf")
    prompt = f"Czy odpowiedź '{answer}' na pytanie '{question}' jest poprawna w kontekście: {context}?"
    evaluation = evaluator(prompt)
    return evaluation

# Tworzenie FAISS Index
model = SentenceTransformer("all-MiniLM-L6-v2")
index, reviews = build_faiss_index(dataset, model)

AttributeError: module 'faiss' has no attribute 'StandardGpuResources'

In [13]:
!pip install faiss-gpu

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
user_query = "What is the best movie to watch with friends?"
response, context = handle_query_with_rag(user_query)

# Ocena odpowiedzi
evaluation = evaluate_response_with_llama(user_query, context, response)
print("Odpowiedź:", response)
print("Ocena:", evaluation)